In [1]:
df = spark.read.option("multiline", "true").json("Files/bing-latest-news.json")


StatementMeta(, 8ed8e71b-f1f7-4744-afe0-1350e936263a, 3, Finished, Available, Finished)

In [2]:
df=df.select("value")

StatementMeta(, 8ed8e71b-f1f7-4744-afe0-1350e936263a, 4, Finished, Available, Finished)

In [3]:
from pyspark.sql.functions import explode
import json

# Creating a new data frame with 'json_object' column, of which, each row contains a new json object
df_exploded = df.select(explode(df['value']).alias('json_object'))


# Collecting the JSON objects into a list
json_list = df_exploded.toJSON().collect()


StatementMeta(, 8ed8e71b-f1f7-4744-afe0-1350e936263a, 5, Finished, Available, Finished)

In [4]:
title = []
description = []
category = []
url = []
image = []
provider = []
datePublished = []

for json_str in json_list:
    try:
        article = json.loads(json_str)

        # Checking if 'category' and 'contentUrl' are present in the JSON object
        if article["json_object"].get("category") and article["json_object"].get("image",{}).get("thumbnail",{}).get("contentUrl"): 
            title.append(article['json_object']['name'])
            description.append(article['json_object']['description'])
            category.append(article['json_object']['category'])
            url.append(article['json_object']['url'])
            image.append(article['json_object']['image']['thumbnail']['contentUrl'])
            provider.append(article['json_object']['provider'] [0]['name'])
            datePublished.append(article['json_object']['datePublished'])

    except Exception as e:
        print(f"Error processing JSON Object: {e}")    

StatementMeta(, 8ed8e71b-f1f7-4744-afe0-1350e936263a, 6, Finished, Available, Finished)

In [5]:
from pyspark.sql.types import StructType, StructField, StringType


# Create a list of tuples where each tuple contains the respective elements from title, description, category, url, image, provider, and datePublished lists
data = list(zip(title,description,category,url,image,provider,datePublished))


# Defining the schema for the DataFrame
schema = StructType([
    StructField("title", StringType(), True),
    StructField("description", StringType(), True),
    StructField("category", StringType(), True),
    StructField("url", StringType(),True),
    StructField("image",StringType(), True),
    StructField("provider", StringType(), True),
    StructField("datePublished", StringType(), True)
])

# Creating a DataFrame using the data and schema defined above
df_cleaned = spark.createDataFrame(data, schema = schema)

StatementMeta(, 8ed8e71b-f1f7-4744-afe0-1350e936263a, 7, Finished, Available, Finished)

In [6]:
from pyspark.sql.functions import to_date, date_format

# Converting the 'datePublished' column to 'dd-MMM-yyyy' format
df_cleaned_final = df_cleaned.withColumn("datePublished", date_format(to_date("datePublished"), "dd-MMM-yyyy"))

StatementMeta(, 8ed8e71b-f1f7-4744-afe0-1350e936263a, 8, Finished, Available, Finished)

In [7]:
from pyspark.sql.utils import AnalysisException

try:
    table_name = 'bing_lake_db.tbl_latest_news'
    df_cleaned_final.write.format("delta").saveAsTable(table_name)

except AnalysisException:

    print("Table already exists")  

    df_cleaned_final.createOrReplaceTempView("vw_df_cleaned_final")

    spark.sql(f""" MERGE INTO {table_name} target_table
                   USING vw_df_cleaned_final source_view

                   ON source_view.url = target_table.url

                   WHEN MATCHED AND
                   source_view.title <> target_table.title OR
                   source_view.description <> target_table.description OR
                   source_view.category <> target_table.category OR
                   source_view.image <> target_table.image OR
                   source_view.provider <> target_table.provider OR
                   source_view.datePublished <> target_table.datePublished 

                   THEN UPDATE SET * 
                
                """)  

StatementMeta(, 8ed8e71b-f1f7-4744-afe0-1350e936263a, 9, Finished, Available, Finished)